In [1]:
print("AI with MOHAMMED")

AI with MOHAMMED


In [2]:
# Step1: Create & Setup hugging face API token in Collab

In [4]:
!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.3/347.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 149.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
# Step3: Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
# Step4: Check HF token
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [ ]:
# Optional: Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla T4


In [8]:
# Step5: Setup pretrained DeepSeek-R1

model_name = "NeoAivara/AI_Doctor_V1"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Step6: Setup system prompt
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.

### Query:
{}

### Answer:
<think>{}
"""

In [ ]:
# Step7: Run Inference on the model

# Define a test question
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or
              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)


print(response)


["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or\n              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,\n              what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Answer:\n<think>\nOkay, so we're looking at a 61-year-old woman who's been dealing with

In [ ]:
print(response[0].split("### Answer:")[1])


<think>
Okay, so we're looking at a 61-year-old woman who's been dealing with involuntary urine loss. That's a big clue. She's been losing urine when she coughs or sneezes. That sounds like it's related to the autonomic nervous system, specifically the urethral sphincter. It makes me think about bladder dysfunction, maybe something like a neurogenic bladder.

Now, she's had a gynecological exam and a Q-tip test. The Q-tip test is usually used to check for urethral obstruction. So, if the Q-tip stays in the vagina and doesn't come out easily, that suggests there's some obstruction. This could be due to a urethral lesion or maybe some kind of nerve issue.

But let's not jump to conclusions yet. She didn't mention any nighttime leakage, so that's interesting. It suggests that her bladder isn't completely full at night, or maybe her sphincter is working okay when she's lying down. That could be a hint that her sphincter isn't completely incompetent, so the involuntary loss during activiti

In [ ]:
# Step8: Setup fine-tuning

# Load Dataset
medical_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[:500]", trust_remote_code = True)

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [ ]:
medical_dataset[1]

{'Question': 'A 33-year-old woman is brought to the emergency department 15 minutes after being stabbed in the chest with a screwdriver. Given her vital signs of pulse 110/min, respirations 22/min, and blood pressure 90/65 mm Hg, along with the presence of a 5-cm deep stab wound at the upper border of the 8th rib in the left midaxillary line, which anatomical structure in her chest is most likely to be injured?',
 'Complex_CoT': "Okay, let's figure out what's going on here. A woman comes in with a stab wound from a screwdriver. It's in her chest, upper border of the 8th rib, left side, kind of around the midaxillary line. First thought, that's pretty close to where the lung sits, right?\n\nLet's talk about location first. This spot is along the left side of her body. Above the 8th rib, like that, is where a lot of important stuff lives, like the bottom part of the left lung, possibly the diaphragm too, especially considering how deep the screwdriver went.\n\nThe wound is 5 cm deep. Tha

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [ ]:
### Finetuning
# Updated training prompt style to add </think> tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""





In [ ]:
# Prepare the data for fine-tuning

def preprocess_input_data(examples):
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]

  texts = []

  for input, cot, output in zip(inputs, cots, outputs):
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)

  return texts # Return the list of strings directly

In [ ]:
finetune_dataset = medical_dataset.map(preprocess_input_data, batched = True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
finetune_dataset["texts"][0]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is sw

In [ ]:
# Step9: Setup/Apply LoRA finetuning to the model

model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = finetune_dataset,
    dataset_text_field = "texts",
    max_seq_length = max_sequence_length,
    dataset_num_proc = 1,
    formatting_func = preprocess_input_data, # Add this line

    # Define training args
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Add this before creating the trainer
if hasattr(model, '_unwrapped_old_generate'):
    del model._unwrapped_old_generate

In [ ]:
trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = finetune_dataset,
    dataset_text_field = "texts",
    max_seq_length = max_sequence_length,
    dataset_num_proc = 1,
    formatting_func = preprocess_input_data, # Add this line

    # Define training args
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir = "outputs",
    ),
)

In [ ]:
# Setup WANDB
from google.colab import userdata
wnb_token = userdata.get("WANDB_API_TOKEN")
# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Medical-CoT-Dataset',
    job_type="training",
    anonymous="allow"
)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammed-kassoiri (mohammed-kassoiri-nethunter) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [ ]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.932200
20,1.424800
30,1.406800
40,1.364700
50,1.391200
60,1.370500


In [ ]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▁▂▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,1.6686758448119808e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.25621
train/learning_rate,0.0
train/loss,1.3705


In [ ]:
# Step10: Testing after fine-tuning
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response)

["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing\n              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,\n              what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Answer:\n<think>\nAlright, let's think about this. This woman is 61 and has been dealin

In [ ]:
print(response[0].split("### Answer:")[1])


<think>
Alright, let's think about this. This woman is 61 and has been dealing with involuntary urine loss for a long time, especially when she coughs or sneezes. That's a classic sign of urinary incontinence, probably due to an overactive bladder. 

Now, she's done a Q-tip test, which is often used to check for urethral obstruction. If the Q-tip stays at 30 cm or higher, that usually means there's some obstruction or a urethral issue. If it's lower, maybe not. 

Given that she's been having these involuntary losses, we're probably dealing with an overactive bladder. Now, let's think about the possibilities for her bladder function. 

Overactive bladder often means the bladder muscle is contracting too much, leading to involuntary contractions. But sometimes, the issue might be with the nerve supply or the muscles not relaxing properly. 

Now, if we look at the cystometry results, what are they telling us? They'll show how much the bladder fills up and how it empties. If there's a lot

In [ ]:
question = """A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue,
              and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative,
              gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium.
              What is the most likely predisposing factor for this patient's condition?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])




<think>
Okay, let's think about this. So we have a 59-year-old man with some pretty classic symptoms of an infection: fever, chills, night sweats, and feeling really tired. These symptoms usually make me think of something like tuberculosis or maybe an infection with a gram-positive bacteria. Oh, and there's a vegetation on the aortic valve. That really catches my attention. I know that a vegetation on the aortic valve is a big clue for something like endocarditis.

Now, the blood cultures are showing gram-positive, catalase-negative, gamma-hemolytic cocci in chains. Hmm, those characteristics are a bit specific. Let me recall what these bacteria are. Oh, right! These are the characteristics of Streptococcus mitis, sometimes also called Streptococcus viridans. They don't grow in a high salt solution, which is interesting because it's a bit different from other streptococci.

Alright, so now, what's the connection between Streptococcus mitis and a vegetation on the aortic valve? I reme

In [ ]:
# Test the fine-tuned model with a new question

question = """What are the common symptoms of pneumonia?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])




<think>
Okay, let's think about pneumonia. It's a lung infection, right? So, what symptoms would you expect? Well, the most obvious one is a cough. Yeah, that makes sense because the lungs are infected, so coughing up phlegm is pretty common. 

Then, there's fever. That's a classic sign of any infection, not just pneumonia. It's a bit tricky because some people might not always show a fever, but it's a good one to remember.

Oh, and shortness of breath! That's definitely a big one. If someone is struggling to breathe deeply, pneumonia is a likely culprit. It makes sense because the lungs are working hard, and the infection is making it hard for them to get enough oxygen.

What else? I remember hearing about chest pain, but it's not as straightforward. It can happen, but it's not as common as the others. It might be more noticeable in people who have pneumonia, especially if it's more serious.

And don't forget about sputum production. Yeah, that's the mucus or phlegm that you cough up

# Task
Deploy the model "MOHAMMED7M7/AI_Doctor_V1" to Hugging Face.

## Save the model

### Subtask:
Save the fine-tuned model and tokenizer to your local environment.


**Reasoning**:
Save the fine-tuned model and tokenizer to a local directory.



In [ ]:
model_lora.save_pretrained("AI_Doctor_V1")
tokenizer.save_pretrained("AI_Doctor_V1")

('AI_Doctor_V1/tokenizer_config.json',
 'AI_Doctor_V1/special_tokens_map.json',
 'AI_Doctor_V1/chat_template.jinja',
 'AI_Doctor_V1/tokenizer.json')

## Login to hugging face

### Subtask:
Login to your Hugging Face account using the `huggingface_hub` library.


**Reasoning**:
Login to Hugging Face using the provided token.



In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
login(hf_token)

## Create a new repository

### Subtask:
Create a new repository on Hugging Face to host your model.


**Reasoning**:
Import the create_repo function, define the repository name, and create the repository on Hugging Face.



In [ ]:
from huggingface_hub import create_repo

repo_name = "MOHAMMED7M7/AI_Doctor_V1"
create_repo(repo_name, exist_ok=True)

print(f"Attempted to create or ensure existence of repository: {repo_name}")

Attempted to create or ensure existence of repository: MOHAMMED7M7/AI_Doctor_V1


## Push the model to the repository

### Subtask:
Push the saved model and tokenizer to the newly created Hugging Face repository.


**Reasoning**:
Import the necessary class and push the model to the Hugging Face Hub.



In [ ]:
repo_name = "MOHAMMED7M7/AI_Doctor_V1"

# Push the model and tokenizer to the Hub
model_lora.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

Saved model to https://huggingface.co/MOHAMMED7M7/AI_Doctor_V1


No files have been modified since last commit. Skipping to prevent empty commit.


## Verify deployment

### Subtask:
Check the Hugging Face repository to ensure the model is uploaded correctly and is ready for inference.


**Reasoning**:
Construct and print the Hugging Face repository URL to verify the uploaded files.



In [ ]:
repo_name = "MOHAMMED7M7/AI_Doctor_V1"
hf_repo_url = f"https://huggingface.co/{repo_name}"
print(f"Please verify the uploaded model files at the following URL: {hf_repo_url}")

Please verify the uploaded model files at the following URL: https://huggingface.co/MOHAMMED7M7/AI_Doctor_V1


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from google.colab import userdata

# Load model and tokenizer
model_name = "NeoAivara/AI_Doctor_V1"  # Replace with actual model path
hf_token = userdata.get('HF_TOKEN').strip() # Get the Hugging Face token and strip whitespace

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token) # Pass the token
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, token=hf_token) # Pass the token

# Prepare input
question = "What are the common symptoms of pneumonia?"
prompt = f"### Question: {question}\n### Answer:"

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to("cuda") # Move inputs to CUDA device if available
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer = response.split("### Answer:")[-1].strip()
print(answer)

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

A. Fever
B. Cough
C. Difficulty breathing
D. Shortness of breath
E. Sputum production
### Response:
<think>
Okay, so I'm trying to figure out what the common symptoms of pneumonia are. I know pneumonia is a lung infection, and it can be pretty serious sometimes, but let's start with the basics. I've heard that pneumonia can cause a cough. Yeah, that seems familiar. There's definitely a cough involved. 

Now, when I think about pneumonia, I also remember that it can make you feel really unwell. People often feel like they have a fever when they have pneumonia. That makes sense because infections like pneumonia can cause the body to react by raising the temperature. So, fever is definitely a common symptom.

Then, I think about how pneumonia affects the lungs. It's an infection, so it can get really uncomfortable to breathe. Sometimes people with pneumonia have trouble catching their breath or feel out of breath. That sounds like difficulty breathing. 

Also, there's sputum production. I